In [65]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from Regression import linear_regression
help(linear_regression)
lr = linear_regression()

Help on class linear_regression in module Regression:

class linear_regression(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  get_confidence_intervals(self, alpha=0.05)
 |  
 |  get_parameters(self, pr=True)
 |      get parameters, standard errors, t-values, p-values
 |  
 |  get_predict(self, X_test, add_const=True)
 |  
 |  get_r_square(self)
 |      return R2 and Adj-R2
 |  
 |  get_residuals(self, normalize=False)
 |      normalize: Default False, return normalized residuals to have unit variance if True
 |  
 |  glm_regression(self, X, y, mod_family, add_const=True)
 |      mod_family: sm.families.Binomial([link]), Gamma(), Gaussian(),InverseGaussian(), NegativeBinomial(), Poisson(), Tweedie()
 |      link: CDFlink, CLogLog, Log, Logit, NegativeBinomial([alpha]), Power([power]), cauthy(), cloglog, identity(), inverse_power(), inverse_sqared(), log, logit, nbinom([alphal]), probit([d

In [115]:
class Robustness:
        
    def stars(self, p):
        if p <= 0.001:
            return '***'
        elif p <= 0.05:
            return '**'
        elif p <= 0.1:
            return '*'
        else:
            return ''
    
    def double_sort(self, X, y, group_names, ngroup=5, take_in_reg = False):
        """
        X: contains cat_names
        take_in_reg: whether take the group_names into regression or not, Default False -> treate like traditional Fama Model alpha 
        group_names: list of two strings, first name will be show on the index, second name will be show on the column
        sort the regression residuals by two cat_names, compare n (biggest) vs 1 (smallest) group by t-test
        """
        X_cols = list(X.columns)
        if not take_in_reg:
            for group in group_names:
                X_cols.remove(group)
        lr.ols_fit(X[X_cols], y, if_print=False)
        resid = lr.get_residuals()
        XX = pd.concat([X[group_names], pd.Series(resid, name='residual', index=X.index)], axis=1)
        for group in group_names:
            XX[group + '_group'] = pd.qcut(XX[group].rank(method='first'), ngroup, labels=False) + 1 # 1-smallest, 5-biggist
        ds_df = pd.pivot_table(XX, values='residual', columns=group_names[1] + '_group', index=group_names[0] + '_group',aggfunc='mean')
        
        test_0 = ds_df.loc[5,:] - ds_df.loc[1,:] # test for first group_name, will add as the last raw
        test_1 = ds_df.loc[:,5] - ds_df.loc[:, 1] # test for first group_name, will add as the last column       
        
        XX_group = XX.groupby([group+'_group' for group in group_names])
        test_0_stars = ["{:.4f}".format(test_0[i]) + self.stars(ttest_ind(XX_group.get_group((1, i))['residual'], XX_group.get_group((5, i))['residual'])[1]) for i in range(1,6)] 
        test_1_stars = ["{:.4f}".format(test_1[i]) + self.stars(ttest_ind(XX_group.get_group((i, 1))['residual'], XX_group.get_group((i, 5))['residual'])[1]) for i in range(1,6)]
        
        ds_df = pd.concat([ds_df, pd.DataFrame({group_names[0] + ' (5-1)':test_0_stars}, index=ds_df.columns).T], axis=0)
        ds_df = pd.concat([ds_df, pd.DataFrame({group_names[1] + ' (5-1)':test_1_stars}, index=ds_df.columns)], axis=1)
        ds_df = ds_df.rename(index={1: '1 (smallest)', 5: '5 (biggest)'}, columns={1: '1 (smallest)', 5: '5 (biggest)'})
        return ds_df        

    
    def regression_result(self, X, y):          
        lr.ols_fit(X, y, if_print=False)
        pa, s, t, pv = lr.get_parameters(pr=False)
        res = pd.DataFrame({'paras': pa, 'pvalues': pv}, index=['intercept'] + list(X.columns))
        res['paras'] = res.apply(lambda x: "{:.4f}".format(x['paras']) + self.stars(x['pvalues']), axis=1)
        r2, ar2 = lr.get_r_square()
        res_r2 = pd.Series([r2, ar2], index=['R2', 'Adj-R2'], name='paras')
        res = pd.concat([res['paras'], res_r2], sort=False, axis=0)
        return res
        
    def cross_effects(self, X, y, keyvar_name, dummy_names):
        """
        X: contains all the dummys
        for the key variate, test if there exists cross effect for different dummies
        """
        X_cols_orig = list(X.columns)
        X_cols = [keyvar_name]
        X_cols_others = X_cols_orig.copy()
        X_cols_others.remove(keyvar_name)
        for dummy in dummy_names:
            X_cols_others.remove(dummy)
        res_all = pd.DataFrame([])
        
        res = self.regression_result(X[X_cols_others + [keyvar_name]], y)
        res = res.rename('Base')
        res_all = pd.concat([res_all, res], sort=False, axis=1)
        
        for dummy in dummy_names:
            cross_name = keyvar_name + ' x ' + dummy
            X[cross_name] = X[keyvar_name] * X[dummy]
            X_cols = X_cols + [dummy, cross_name]
            res = self.regression_result(X[X_cols_others + [keyvar_name, dummy, cross_name]], y)
            res = res.rename('ADD: ' + dummy)
            res_all = pd.concat([res_all, res], sort=False, axis=1)
        return res_all.loc[X_cols + X_cols_others + ['intercept', 'R2', 'Adj-R2']]       
    
    
    def stepwise_regression(self, X, y, X_cols):
        """
        conduct stepwise tests for X in sequence X_cols
        """    
        print('Stepwise test for ' + y.name + ': ' + '-> '.join(X_cols))
        for i in range(len(X_cols)):
            reg_X_cols = X_cols[:i+1]
            res = self.regression_result(X[reg_X_cols], y)
            res = res.rename(i+1)
            res_all = pd.concat([res_all, res], sort=False, axis=1)
        return res_all.loc[reg_X_cols + ['intercept', 'R2', 'Adj-R2']]

## Sample

In [23]:
filename = 'airbnb.csv'
data = pd.read_csv(filename)
print(len(data))
data.head().T

74111


,0,1,2,3,4
id,6901257,6304928,7919400,13418779,3808709
log_price,5.01,5.13,4.98,6.62,4.74
property_type,Apartment,Apartment,Apartment,House,Apartment
room_type,Entire home/apt,Entire home/apt,Entire home/apt,Entire home/apt,Entire home/apt
amenities,"{""Wireless Internet"",""Air conditioning"",Kitche...","{""Wireless Internet"",""Air conditioning"",Kitche...","{TV,""Cable TV"",""Wireless Internet"",""Air condit...","{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...","{TV,Internet,""Wireless Internet"",""Air conditio..."
accommodates,3,7,5,4,2
bathrooms,1,1,1,1,1
bed_type,Real Bed,Real Bed,Real Bed,Real Bed,Real Bed
cancellation_policy,strict,strict,moderate,flexible,moderate
cleaning_fee,True,True,True,True,True


### Double Sort for Regression Residuals

In [117]:
X_cols = ['bedrooms', 'bathrooms', 'latitude', 'review_scores_rating']
y_col = 'log_price'

subdata = data[X_cols + [y_col]].dropna()
y = subdata[y_col]
X = subdata[X_cols]

In [116]:
r = Robustness()
r.double_sort(X, y, ['bedrooms', 'review_scores_rating'], ngroup=5)

,1 (smallest),2,3,4,5 (biggest),review_scores_rating (5-1)
1 (smallest),-0.15,-0.0675,0.00644,-0.0268,0.187,0.3371***
2,-0.315,-0.213,-0.165,-0.118,-0.149,0.1652***
3,-0.298,-0.223,-0.135,-0.0565,-0.148,0.1504***
4,-0.182,-0.0792,-0.0652,0.149,-0.132,0.0505**
5 (biggest),0.408,0.458,0.486,0.527,0.493,0.0846***
bedrooms (5-1),0.5586***,0.5253***,0.4791***,0.5534***,0.3060***,NaN


### Stepwise Regression

In [5]:
X_cols = ['bedrooms', 'bathrooms', 'latitude', 'review_scores_rating', '']
y_col = 'log_price'

subdata = data[X_cols + [y_col]].dropna()
y = subdata[y_col]
X = subdata[X_cols]

In [6]:
r = Robustness()
r.stepwise_regression(X, y, X_cols)

Stepwise test for log_price: bedrooms-> bathrooms-> latitude-> review_scores_rating


C:\Users\cuixu\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


,1,2,3,4
bedrooms,0.3889***,0.3436***,0.3427***,0.3421***
bathrooms,NaN,0.1181***,0.1240***,0.1239***
latitude,NaN,NaN,0.0071***,0.0077***
review_scores_rating,NaN,NaN,NaN,0.0073***
intercept,4.2591***,4.1713***,3.8937***,3.1820***
R2,0.24,0.247,0.248,0.255
Adj-R2,0.24,0.247,0.248,0.255


### Cross Effect from Dummies

In [10]:
X_cols = ['bedrooms', 'bathrooms', 'latitude', 'review_scores_rating', 'instant_bookable', 'host_has_profile_pic']
y_col = 'log_price'
dummy_names = ['instant_bookable', 'host_has_profile_pic']

subdata = data[X_cols + [y_col]].dropna()
subdata[dummy_names] = (subdata[dummy_names] == 't').astype('int')
y = subdata[y_col]
X = subdata[X_cols]

In [11]:
r = Robustness()
r.cross_effects(X, y, 'bedrooms', dummy_names)

,Base,ADD: instant_bookable,ADD: host_has_profile_pic
bedrooms,0.3423***,0.3455***,0.2557***
instant_bookable,NaN,-0.0514***,NaN
bedrooms x instant_bookable,NaN,-0.0118,NaN
host_has_profile_pic,NaN,NaN,-0.3130**
bedrooms x host_has_profile_pic,NaN,NaN,0.0869
bathrooms,0.1234***,0.1231***,0.1234***
latitude,0.0076***,0.0076***,0.0076***
review_scores_rating,0.0073***,0.0071***,0.0073***
intercept,3.1831***,3.2228***,3.4952***
R2,0.255,0.257,0.255
